**Part 1: Analyze and Explore the Climate Data**

In this section, you’ll use Python and SQLAlchemy to do a basic climate analysis and data exploration of your climate database. Specifically, you’ll use SQLAlchemy ORM queries, Pandas, and Matplotlib. To do so, complete the following steps:

1) Note that you’ll use the provided files (climate_starter.ipynb and hawaii.sqlite) to complete your climate analysis and data exploration.
2) Use the SQLAlchemy create_engine() function to connect to your SQLite database.
3) Use the SQLAlchemy automap_base() function to reflect your tables into classes, and then save references to the classes named station and measurement.
4) Link Python to the database by creating a SQLAlchemy session.
    Remember to close your session at the end of your notebook.
5) Perform a precipitation analysis and then a station analysis by completing the steps in the following two subsections.

In [1]:
# from climate starter code
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
# from climate starter code
import numpy as np
import pandas as pd
import datetime as dt


In [3]:
# from climate starter code python sql toolkit and ORM
import sqlalchemy

In [4]:
# from climate starter code create engine to hawaii.sqlite
from sqlalchemy import create_engine
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [5]:
from sqlalchemy.ext.automap import automap_base

Base = automap_base()
Base.prepare(autoload_with=engine)
Base.classes.keys()


['measurement', 'station']

In [6]:
Measurement = Base.classes.measurement
Station = Base.classes.station

In [7]:
from sqlalchemy.orm import Session
session = Session(engine)

In [8]:
from sqlalchemy import inspect
inspector = inspect(engine)
columns = inspector.get_columns('Measurement')
for c in columns:
    print(c['name'],c["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [9]:
inspector = inspect(engine)
columns = inspector.get_columns('Station')
for c in columns:
    print(c['name'],c["type"])


id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [10]:
session.close()

**Percipitation Analysis**

1) Find the most recent date in the dataset.
2) Using that date, get the previous 12

In [11]:
from sqlalchemy import func
most_recent_date = session.query(func.max(Measurement.date)).scalar()
print("Most recent date:", most_recent_date)

Most recent date: 2017-08-23


In [12]:
from datetime import datetime, timedelta
